*Stanislav Borysov [stabo@dtu.dk], DTU Management*
# Advanced Business Analytics

## Refreshing Python and Machine Learning: Part 3 - Pandas

*Based on the notebooks from 42184 Data Science for Mobility E19 / 42577 Introduction to Business Analytics E19*

So, in the previous notebook you learned about Numpy. You probably felt it to be sometimes complicated for routine tasks (e.g. get only data from one of the fields of a file, convert individual fields). 

The Pandas module tries to automatize a series of typical tasks that us, Data Scientists, have to do all the time. This doesn't mean that what you learned with Numpy is not useful anymore. In fact, all of that will be precious in many circumstances (e.g. Pandas uses a lot of Numpy features too!).

## 1. Overview: Data frames - a powerful data-structure

The pandas module provides a powerful data-structure called a data frame.

It is similar, but not identical to:
* a table in a relational database,
* an Excel spreadsheet,
* a dataframe in R.

Data frames can be read and written to/from:
* database queries, database tables
* CSV files
* json files

Beware that data frames are memory resident!

## Importing pandas

The pandas module is usually imported with the alias pd.

In [ ]:
import pandas as pd

## Series - DataFrames

Pandas is object-oriented. We create data frames by constructing instances of different classes. The two most important classes are:

* `DataFrame`
* `Series`

Let's create our first `Series`.

**(Don't forget to play with our code to really understand what's going on!)**

In [ ]:
import numpy as np
data = np.random.randn(5)    #create 5 random numbers
data

In [ ]:
my_series = pd.Series(data, index=['a', 'b', 'c', 'd', 'e'])
my_series

We can **plot** a `Series` by invoking the `plot()` method on an instance of a `Series` object. The x-axis will automatically be labelled with the series index!

In [ ]:
%matplotlib inline
my_series.plot()

We can also create a series with automatic index.

In [ ]:
pd.Series(data)

Series can be accessed using the same syntax as arrays and dicts. So, we are going to use the labels in the index to access each element.

In [ ]:
my_series['b']

We can also use the label like an attribute:

In [ ]:
my_series.b

Finally, we can specify a range of labels to obtain a slice:

In [ ]:
my_series[['b', 'c']]

Now, let's create a **DataFrame** by passing a numpy array, with a datetime index and labeled columns.

First, the datetime index:

In [ ]:
dates = pd.date_range('20130101', periods=6)
dates

Now, we create the DataFrame:

In [ ]:
['A', 'B', ...]

In [ ]:
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df

and a second `DataFrame` by passing a dict of objects that can be converted to series-like.

In [ ]:
df2 = pd.DataFrame({ 'A' : [1., 2.1, 3.5, 8.1],
                    'B' : pd.Timestamp('20130102'),
                    'C' : pd.Series(1,index=list(range(4)),dtype='float32')})
print(df2) 

In [ ]:
print(df2.dtypes)

## Viewing Data

You can see the top & bottom rows of the frame:

In [ ]:
# 5 top rows
df.head()

In [ ]:
# 3 last rows
df.tail(3)

Let's display the **index, columns, and the underlying numpy data**:

In [ ]:
df.index

In [ ]:
df['index']

In [ ]:
len(df.columns)

In [ ]:
df.values

If you are interested in seeing quick statistic summary of your data then:

In [ ]:
df.describe()

and if you want to sort your data by a certain column:

In [ ]:
df.sort_values(by='B')

## Selecting data

If you want to select a specific column to print:

In [ ]:
print(df['A lk afg a'])

In [ ]:
df.A

Rows can be selected using either the index:

In [ ]:
print(df['20130102':'20130104'])

or simpler using the numeric index of the rows you want to print:

In [ ]:
print(df[1:4])

For getting a cross section using a label:

In [ ]:
df.loc[dates[0]]

If you want to be even more specific and show only the first two columns of the first three rows:

In [ ]:
df.loc['20130101':'20130103',['A','B']]

Sometimes we do not care about the actual value of the index, but only its relevant position. Therefore, if we want to print the values of the fourth date-row:

In [ ]:
print(df.iloc[3])

By integer slices, acting similar to numpy/python:

In [ ]:
print(df.iloc[3:5,0:2])

Another interesting tool is **the boolean indexing**. We can isolate the rows that meet a specific criterion that we want, such as:

In [ ]:
df.A > 0

In [ ]:
print(df[df.A > 0])

For filtering, the following command helps:

In [ ]:
#First, let's create a copy of our dataframe
df2 = df.copy()

# Now, we will add a new column, named "E" with the following values:
df2['E'] = 'four'

#So, now if we want to show the rows that have the value "two" or "four" at the column "E", we will type: 
df2[df2['E'].isin(['two','four'])]

In [ ]:
a = ['a', 'b', 'c']
b = 'b'
b in a

## Operations 

One of the essential pieces of NumPy is the ability to perform quick elementwise operations, both with basic arithmetic (addition, subtraction, multiplication, etc.) and with more sophisticated operations (trigonometric functions, exponential and logarithmic functions, etc.). Pandas inherits much of this functionality from NumPy, and the universal functions are key to this.

Pandas includes a couple useful twists, however: for unary operations like negation and trigonometric functions, these universal functions will preserve index and column labels in the output, and for binary operations such as addition and multiplication, Pandas will automatically align indices when passing the objects to the universal function. This means that keeping the context of data, and combining data from different sources – both potentially error-prone tasks with raw NumPy arrays – become essentially foolproof with Pandas. We will additionally see that there are well-defined operations between one-dimensional `Series` structures and two-dimensional `DataFrame` structures.

Because Pandas is designed to work with NumPy, any NumPy  universal function will work on pandas Series and DataFrame objects. Lets start by defining a simple Series and DataFrame on which to demonstrate this:

In [ ]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

In [ ]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)), columns=['A', 'B', 'C', 'D'])
df

If we apply a NumPy universal function on either of these objects, the result will be another Pandas object with the indices preserved:

In [ ]:
np.exp(ser)

Or, for a slightly more complex calculation:

In [ ]:
np.sin(df * np.pi / 4)

For binary operations on two `Series` or `DataFrame` objects, Pandas will align indices in the process of performing the operation. This is very convenient when working with **incomplete data**, as we’ll see in some of the examples below.

As an example, suppose we are combinging two different data sources, and find only the top three US states by area and the top three US states by population:

In [ ]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,'New York': 19651127}, name='population')

In [ ]:
area

In [ ]:
population

Let’s see what happens when we divide these to compute the population density:

In [ ]:
population / area

The resulting array contains the **union** of indices of the two input arrays, which could be determined using standard Python set arithmetic on these indices:

In [ ]:
area.index | population.index

Any item for which one or the other does not have an entry is marked by NaN, or “Not a Number”, which is how Pandas marks missing data. This index matching is implemented this way for any of Pythons built-in arithmetic expressions; any missing values are filled-in with NaN by default:

In [ ]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

If filling-in NaN values is not the desired behavior, the fill value can be modified using appropriate object methods in place of the operators. For example, calling `A.add(B)` is equivalent to calling A + B, but allows optional explicit specification of the fill value:

In [ ]:
A.add(B, fill_value=0)

A similar type of alingment takes place for both columns and indices when performing operations on dataframes:

In [ ]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)), columns=list('AB'))
A

In [ ]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),columns=list('BAC'))
B

In [ ]:
A+B

Notice that indices are aligned correctly irrespective of their order in the two objects, and indices in the result are sorted. Similarly to the case of the Series, we can use the associated object’s arithmetic method and pass any desired fill_value to be used in place of missing entries:

In [ ]:
A.add(B, fill_value=np.mean(A.values))

### Merge, Join, Append

Pandas provides various facilities for easily combining together Series, DataFrame, and Panel objects with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

In [ ]:
df = pd.DataFrame(np.random.randn(10, 4))
df

In [ ]:
# break it into pieces
pieces = [df[:3], df[3:7], df[7:]]
pieces[0]

**`pandas.concat`** command concatenates pandas objects along a particular axis with optional set logic along the other axes.

In [ ]:
pd.concat(pieces)

**`DataFrame.merge`** command merges DataFrame objects by performing a database-style join operation by columns or indexes.

In [ ]:
left

In [ ]:
right

In [ ]:
left = pd.DataFrame({'key': ['foo', 'foo'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'foo'], 'rval': [4, 5]})

pd.merge(left, right, on='key')

**`DataFrame.join`** command joins columns with other DataFrame either on index or on a key column. Efficiently Join multiple DataFrame objects by index at once by passing a list.

In [ ]:
left.join(right, lsuffix='_caller', rsuffix='_other')

**`DataFrame.append`** command appends rows of other to the end of this frame, returning a new object. Columns not in this frame are added as new columns.

In [ ]:
df.append(pieces[0], ignore_index=True)

### Grouping in Pandas

Most of the time, you have a set of data that lends itself to being categorized or grouped. As a general example, let's say we have data on a wide variety of people. We may perform an analysis where we compare groups in the data based on age, gender, birth month, shoe size, or birth city; the options are as numerous as the data points!

I think it would be intriguing to work now with a proper dataset from Citi Bike - Daily Ridership and Membership Data

In [ ]:
df = pd.read_csv('data/201707-citibike-tripdata.csv')
df.head()

It looks like there is a good opportunity to break the data down into groups to look for some interesting trends. Some ideas are:

* Group on the gender column and see if there are more male or female riders.
* Do specific stations get used more than others? We can group on the station start or finish id.
* Group the data on the day of the week, to see if there is more utilization for a particular day, on average.

### How about a few examples?

If we want to group by just the gender, then we pass this key (column name) to the groupby function as the sole argument. This example is the simplest form of grouping.

In [ ]:
groupedGender = df.groupby('gender')
print(groupedGender)

The output shows that groupby returns a pandas `DataFrameGroupBy` object. Pandas has just made some internal calculations about the new gender groups and is ready to apply some operation on each of these groups.

Getting back to the data, if we use the `count` method, we can see the total number of entries **for each gender group**. For reference, here's what the website (from where the dataset was downloaded) says for the gender codes - "Gender (Zero=unknown; 1=male; 2=female)"

In [ ]:
groupedGender.size()

In [ ]:
groupedGender.count()

In [ ]:
# look at the size as a percentage of the whole (using the trip)
total = df.gender.count()
groupedGender.size() / total * 100

It looks like males make up the majority of Citi Bike riders (~63%)!

We can use a single column from the `DataFrameGroupBy` object and apply some aggregation function on it - how about the **mean** and **standard deviation** of the trip durations for all three groups?

In [ ]:
groupedGender['tripduration'].mean() / 60.

In [ ]:
# Don't have to use the bracket notation
groupedGender.tripduration.std() / 60.

Although males make up the majority of Citi Bike riders, there's not much of a difference in their trip durations. Interestingly, gender unknown riders take 2x as long of rides on average. These riders are likely single-use customers (when you purchase a one time pass at a Citi Bike kiosk you are not asked for your gender).

So there are some summary statistics for these groups (as an aside, you can use the describe function to get these statistics and more in one call). That's a whole lot of spread around the mean, which probably means there are some outliers in the data (maybe people that kept the bike for days). Just a brief look at this even though it's outside the scope, because I'm sure you were all interested 😊

In [ ]:
df[df.tripduration > 10000].tripduration.count()

Our suspicions are confirmed - there are many bike rentals outside 2:45 even though the "max" is supposed to be 30 minutes (or 45 if you're a Citi Bike member).

In [ ]:
from ggplot import *

df_short = df[df.tripduration < 10000]
df_short.tripduration = df_short.tripduration / 60.
ggplot(df_short, aes(x='tripduration')) + geom_histogram(bins=30) + xlab("Trip Duration (mins)") + ylab("Count")

One last example is looking at which are the five favorite start and end stations. We'll group the data based on the start and end station names, apply the count function, and sort the values is descending order.

In [ ]:
groupedStart = df.groupby('start station name')
groupedStart['start station name'].count().sort_values(ascending=False)[:5]

In [ ]:
groupedEnd = df.groupby('end station name')
groupedEnd['end station name'].count().sort_values(ascending=False)[:5]

One last thing! Whenever we have a time-series dataset it is very useful to assign the time as index for our dataframe. Let's do that with the method **`set_index()`** of the DataFrame object.

In [ ]:
df=df.set_index('starttime')
df.head()

You have already seen today how we could identify demand patterns using the timestamps that the dataset offers. Next time, we will be able to see more using the Python Pandas module.

## 2. Exercises

Let's explore in more depth the possibilities providaded by the Python Pandas module. To make things less confusing, we decided to use the same dataset as in the numpy exercise. In fact, you'll start with doing exactly the **same** exercises. Then, an incremental set of exercises end up with ... !!!

So, let's start. Since we are going to work with Pandas, let's just import it, shall we (you'll eventually also need Numpy, so import that too)?

In [ ]:
# ...

We need some data. Please open the file "pickups_zone_1_15min.csv". This corresponds to the series of taxi-pickups in New York zone 1 (an area in the Manhattan island). 

You can use the method read_csv(file), which **returns** a DataFrame:

> df=pd.read_csv(file)

df is a variable that now has a Pandas DataFrame


In [ ]:
# ...

Good, so we just loaded the file, now let's look at its content (and keep remembering how more complicated it was in Numpy). 

In [ ]:
# ...

Just to be sure, let's check how many lines the file actually has (now it should be 262848, one line less, why?). 

In [ ]:
# ...

Print the first 10 lines of this file.

In [ ]:
# ...

Oh, remember that we need to make a single field with the datetime (instead of 3 separate ones...)!

Pandas actually allows us to simplify that right when we load the file. So, let's do it again, but now take a look at the following Stackexchange thread: https://stackoverflow.com/questions/38509512/pandas-read-csv-with-date-in-2-columns

How about applying this here?



In [ ]:
# ...

It may happen that the (new) temporal field is still a string and not a datetime object. Can you correct that?

In [ ]:
# ...

In fact, it can be useful to assign the time as index for our dataframe. Let's do that with the method set_index() of the DataFrame object. Notice that it doesn't do "in-place" changes, in other words, this command would not work: 
> df.set_index(KEY)

You'd have to do

>df=df.set_index(KEY)

In [ ]:
# ...

Important note: You can always get a vector with the indeces themselves with 
> df.index

To understand this, just try it yourself

So, just for fun (and to compare with the amount of work we did with Numpy), let's get **only** the pickups part. 

In [ ]:
# ...

Trivial, isn't it? It's even indexed correctly! :-)

Now, it's time for our histogram for the pickups. If you can't remember or don't know how it's done, feel free to search online (something like "Pandas histogram")

again, don't forget to add

>import matplotlib.pyplot as plt
>
>%matplotlib inline



In [ ]:
# ...

It's even scary how easy it is, right?

What about the scatter plot? It won't be as direct, you will have to use the pyplot one (the one you used last time)

In [ ]:
# ...

Now let's do that version with the colors. As last time, you need to create a new vector with the minutes since midnight.

In [ ]:
# ...

Even though it may sound redundant, it may be useful to add this new series to the DataFrame (as a new column, in practice).

In [ ]:
# ...

So, let's do the graph with the colors. 

Remember, imagine that the number of minutes since midnight (that you just created) corresponds to a color. The function scatter allows you to give this list straight away and plot it (just use the argument c, for example "c=my_minute_since_midnight_list". 

Do you want to try?

In [ ]:
# ...

Now, for the 24-hr average plot, where the x axis is 0 to 1440 (1440=24 hours X 60 minutes), and you show the average per minute.

(don't forget to add the 5 and 95 quantiles) 


A small tip (mean of 15 minutes of the day):

> df.loc[df['minute_of_day']==15].mean()
>
>
Output:

pickups          205.344047

minute_of_day     15.000000

dtype: float64


**IMPORTANT:** notice that the output above is not just the mean of the _pickups_ (which is what we want). It also returns the mean of the minute_of\_day itself (which is obviously 15!)... This may confuse things later on, so how do you get the mean of pickups only?



In [ ]:
# ...

### Correlations

_For this part, we'll keep the text as is. The exercises are exactly the same. _

One very important task in Data Science modeling is to find (and understand) correlations between different variables. Let's do a few simple exercises.

Let's start with a simple question: are the different areas correlated between them? If yes, it may be interesting knowledge. For example, maybe we can share data between them to predict better, later.


Tip: Try to make a single DataFrame with ALL time series (s1, s17, s21, s28). 


In [ ]:
def my_read_csv(file):
    df = pd.read_csv(file, sep=",", parse_dates={'dt': [0, 1, 2]})
    df['datetime'] = df['dt'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H %M"))
    del df['dt']
    df = df.set_index('datetime')
    return df

In [ ]:
s1 = my_read_csv('data/pickups_zone_1_15min.csv')
s17 = my_read_csv('data/pickups_zone_17_15min.csv')

In [ ]:
s17.head()

In [ ]:
smerge = s1.join(s17, rsuffix="17")
smerge['pickups1'] = smerge['pickups']
del smerge['pickups']
smerge.head()

In [ ]:
smerge.corr()#['pickups1']['pickups17']

Now, a more interesting question: are there correlations between a given area, and the other areas in earlier time steps? 

This is a VERY important one. If you find high correlation, for example, between area 1 at time t, with area 17 at time t-1, then you can use area 17 to predict area 1!

To check this, you need to play a little bit with the vectors. Let's call a vector that is shifted in time for 1 time step, a "lag1" vector. 

The process is similar to Numpy's, but there's a few tweaks. To make things simpler, we copy a solution from Stackoverflow:

In [ ]:
def buildLaggedFeatures(s, columns, lag=1, dropna=True):
    '''
    From http://stackoverflow.com/questions/20410312/how-to-create-a-lagged-data-structure-using-pandas-dataframe
    Builds a new DataFrame to facilitate regressing over all possible lagged features
    '''
    if type(s) is pd.DataFrame:
        new_dict={}
        for c in s.columns:
            new_dict[c]=s[c]
        for col_name in columns:
            new_dict[col_name]=s[col_name]
            # create lagged Series
            for l in range(1,lag+1):
                new_dict['%s_lag%d' %(col_name,l)]=s[col_name].shift(l)
        res=pd.DataFrame(new_dict,index=s.index)

    elif type(s) is pd.Series:
        the_range=range(lag+1)
        res=pd.concat([s.shift(i) for i in the_range],axis=1)
        res.columns=['lag_%d' %i for i in the_range]
    else:
        print('Only works for DataFrame or Series')
        return None
    if dropna:
        return res.dropna()
    else:
        return res 

In [ ]:
# ...

Let's now check those correlations

In [ ]:
# ...

WOW! Very interesting!! This means that you can use data from these other areas to predict for area 1... This is useful when there is missing data in area 1, for example... 

Now, let's get to the autocorrelogram. Well, in Pandas, this is trivial!

You can use the auto_corr method:

>s1=df['pickups'] #s1 is now a Series
>
>print(s1.autocorr(1))  #gives the autocorrelation of lag 1

Output:

0.97670270955196936


Cool, now you just need to get values for the different lags (1, 2, 3, 4...) and plot the values!

In [ ]:
# ...

### Categorical variables, combining datasets

Ok, you're almost ready to start rockin' with actual Data Science learning :) 

There's a last thing before that. Quite often, the data that you get is not in numerical form. Two obvious examples are time partitioning, like weekdays ("Monday", "Tuesday"...) or time of day ("Morning rush hour", "lunch time"). These are in fact words, how can we use them in our modeling, if in practice it always requires numerical quantities?

Particularly with mobility data, the examples above are quite common. For example, the time of day (rush hour VS low demand times) can be crucial for prediction!

Let's start by creating a new column in our dataframe with the day of week (we'll do it for you):

In [ ]:
smerge['weekday']=[d.weekday() for d in smerge.index]
#for each element in the index use the "weekday" function 
#(remember that the index is the datetime series)

To make sure that it worked just take a look at the resulting DataFrame

In [ ]:
smerge.head()

Wow, these are already numbers, but wait!... Do those quantities mean actually _something_ or aren't they just individual symbols for the week days?

Other ways to respond to this question: if instead of these numbers you used others (e.g. starting at 1 instead of 0?) wouldn't the result be the same? If you use algebra on it, does the result make sense (Thursday-Tuesday=Wednesday...). 

The bottom line is that, in general, you shouldn't use these numbers directly in a regression model because week days do not correspond to quantities. This is a **categorical** variable, which can have one of a finite set of values. In our case:

_weekday_ $\in $ {Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday}

So, let's convert those values into something useful. A first obvious thing to do is to get whether a day is a week day or weekend. Let's define a simple function for it:

In [ ]:
def is_weekend(weekday):
    return weekday in [5, 6]

You can test it...

Another nice thing will be to actually put names there. Here's another function

In [ ]:
def weekday_word(weekday):
    if weekday==0:
        return "Monday"
    elif weekday==1:
        return "Tuesday"
    elif weekday==2:
        return "Wednesday"
    elif weekday==3:
        return "Thursday"
    elif weekday==4:
        return "Friday"
    elif weekday==5:
        return "Saturday"
    else:
        return "Sunday"

Now, let's create the new columns in our DataFrame!

In [ ]:
smerge.info()

In [ ]:
smerge['is_weekend'] = smerge['weekday'].apply(is_weekend)
smerge['wd'] = [weekday_word(d) for d in smerge['weekday']]
smerge.head()

Take a look at the new DataFrame. Seems more useful, right? 

In [ ]:
# ...

Let's get the "time of day" information. Again, we provide you with the corresponding function. 

In [ ]:
def time_of_day(hour):
    if hour<7:
        return "night"
    elif hour<9:
        return "morning rush"
    elif hour<12:
        return "morning"
    elif hour<14:
        return "lunch time"
    elif hour<17:
        return "afternoon"
    elif hour<20:
        return "afternoon rush"
    elif hour<23:
        return "evening"
    else:
        return "night"
    
# In fact, we created another function that has the SAME functionality as above just to show you how compact and elegant 
# Python can be. 
# Don't worry, you're not expected to reach this level before of a lot of experience. 
def time_of_day2(hour):    
    h_thresholds=[7,8,12,14,17,20,23]
    h_names=["night", "morning rush", "morning", "lunch time", "afternoon", "afternoon rush", "evening", "night"]
    return h_names[next(x[0] for x in enumerate(h_thresholds) if x[1] >hour)]

Can you create the new column with time of day?

In [ ]:
# ...

The other **very important** technique is to transform categorical variables into "dummy variables". A dummy variable is typically binary (so it's either 1/0, or True/False), and corresponds to one single possible categorical value. 

**So, you transform a single variable with N different values **

_time of day -> {night, morning rush, morning...evening}_

**into N dummy variables, each one with 2 values (1 or 0):**

_night -> {0, 1}_

_morning rush -> {0, 1}_

_..._

_evening -> {0, 1}_


Yes, we know it sounds like hard work, but in Pandas, it's all done trivially:

In [ ]:
sm_dummies=pd.get_dummies(smerge, columns=['tod'])  #this creates dummies for the "time_of_day" variable. 

Finally, a well deserved reward for you. Just run the method describe() in your DataFrame and enjoy!... 

>s.describe()
>

Output:

_something useful_  :-)


Think about how d

In [ ]:
# ...

In [ ]:
#sm_dummies.to_csv("filename") # save dummies to a file

## 3. Homework: Data Fusion - Data Preparation

**Data preparation** (or data preprocessing) in this context means manipulation of data into a form suitable for further analysis and processing. It is a process that involves many different tasks and which cannot be fully automated. Many of the data preparation activities are routine, tedious, and time consuming. It has been estimated that data preparation accounts for 60%-80% of the time spent on a data mining project!

**Data fusion** is the process of integrating multiple data sources to produce more consistent, accurate, and useful information than that provided by any individual data source. 

In this notebook, we are going to work again with the NYC Taxi dataset. We will add a second interesting dataset with weather information, and we will try to produce more useful information for our already formulated prediction models (Week 3 Lecture).

*Note: We will also use a new visualization library called "seaborn". You will need to install the package "seaborn" from anaconda's navigator.*

In [ ]:
#Import necessary libraries
import pandas as pd
from pandas import DataFrame, read_csv

import matplotlib
import matplotlib.pyplot as plt
# Enable inline plotting
%matplotlib inline
# matplotlib style options
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 8)

import datetime

import numpy as np
from collections import defaultdict

# Python visualization library
import seaborn as sns

import calendar

import numpy
from numpy import nan

During the previous lecture we formulated and used a function that reads the necessary csv files at a proper format. We will use it again here:

In [ ]:
def my_read_csv(file):
    df = pd.read_csv(file, sep=',', parse_dates={'dt': [0, 1, 2]})
    df['datetime']=df['dt'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H %M"))
    del df['dt']
    return df.set_index('datetime')

In [ ]:
# Remember to change the path below based on where you have chosen to save your files
s1=my_read_csv("data/pickups_zone_1_15min.csv")
s17=my_read_csv("data/pickups_zone_17_15min.csv")
s21=my_read_csv("data/pickups_zone_21_15min.csv")
s28=my_read_csv("data/pickups_zone_28_15min.csv")

smerge=s1.join(s17, rsuffix="17")
smerge=smerge.join(s21, rsuffix="21")
smerge=smerge.join(s28, rsuffix="28")
smerge['pickups1']=smerge['pickups']
del smerge['pickups']
smerge.head()

In class's notebook, we successfully found some interesting correlations for example, between area 1 at time t, with area 17 at time t-1. We will use them again here:

In [ ]:
# Solution from Stackoverflow as used before
def buildLaggedFeatures(s,columns, lag=1, dropna=True):
    '''
    From http://stackoverflow.com/questions/20410312/how-to-create-a-lagged-data-structure-using-pandas-dataframe
    Builds a new DataFrame to facilitate regressing over all possible lagged features
    '''
    if type(s) is pd.DataFrame:
        new_dict={}
        for c in s.columns:
            new_dict[c]=s[c]
        for col_name in columns:
            new_dict[col_name]=s[col_name]
            # create lagged Series
            for l in range(1,lag+1):
                new_dict['%s_lag%d' %(col_name,l)]=s[col_name].shift(l)
        res=pd.DataFrame(new_dict,index=s.index)

    elif type(s) is pd.Series:
        the_range=range(lag+1)
        res=pd.concat([s.shift(i) for i in the_range],axis=1)
        res.columns=['lag_%d' %i for i in the_range]
    else:
        print('Only works for DataFrame or Series')
        return None
    if dropna:
        return res.dropna()
    else:
        return res 

In [ ]:
smerged_lagged_s1=buildLaggedFeatures(smerge, ['pickups1', 'pickups17', 'pickups21', 'pickups28'])

In [ ]:
smerged_lagged_s1.head()

## Weather Data

This public dataset was created by the National Oceanic and Atmospheric Administration (NOAA) and includes global data obtained from the USAF Climatology Center. This dataset covers GSOD data between 1929 and 2016, collected from over 9000 stations.

https://cloud.google.com/bigquery/public-data/noaa-gsod

In [ ]:
dfWeather=pd.read_csv("data/gsod2016.csv")

In [ ]:
dfWeather.head()

In [ ]:
list(dfWeather)

The dataset includes weather data from the whole world. We should find a way to select only the weather data referring to Manhattan.

As you can see from the list of attributes above, there is a key that could help us select what we need; the **stn** key, that represents the station ID from which each record has been collected. 

But since an ID number does not allow us to know to which area it refers to, we will use a second dataset with the list of all stations and their location details.

In [ ]:
stations=pd.read_csv("data/stations.csv")

In [ ]:
len(stations)

In [ ]:
stations.tail()

In [ ]:
stations.head()

In [ ]:
stations.describe()

We cannot see cleary which stations refer to Manhattan, therefore we will filter the dataset using the lat-lon columns.

### Manhattan Isolation using lat-lon

In [ ]:
# Selection of values based on openstreetmap website export tools
# http://www.openstreetmap.org/export#map=11/40.7504/-73.9359

minlong = -74.04
minlat = 40.69
maxlong = -73.91
maxlat = 40.82

In [ ]:
masklong = (stations['lon'] >= minlong) & (stations['lon'] <= maxlong)
stationsLon = (stations.loc[masklong])

masklat = (stationsLon['lat'] >= minlat) & (stationsLon['lat'] <= maxlat)
stationsLonLat = (stationsLon.loc[masklat])

Now that we filtered the dataset, let's see how many stations we have...

In [ ]:
len(stationsLonLat)

Well... Something goes wrong here... Let's see how many stations are actually located on Earth!!

In [ ]:
minlong1 = -180
minlat1 = -90
maxlong1 = 180
maxlat1 = 90

In [ ]:
masklong = (stations['lon'] >= minlong1) & (stations['lon'] <= maxlong1)
stationsLon1 = (stations.loc[masklong])

masklat = (stationsLon1['lat'] >= minlat1) &(stationsLon1['lat'] <= maxlat1)
stationsLonLat1 = (stationsLon1.loc[masklat])

In [ ]:
len(stationsLonLat1)

In [ ]:
plt.plot(stationsLonLat1.lon)

Only 18 stations have reasonable lat-lon values... Therefore, we cannot do much using those attributes. Let's see if we can use the column **state**.

In [ ]:
stations.loc[stations['state']=='NY'].head()

As you can see, there are stations in NY. Can you see which is the problem and why we could not see them before?

It seems that the information of decimals from the lat-lon columns has been lost. We can restore it by making the following division:

In [ ]:
stations.loc[:,'lat'] /= 1000
stations.loc[:,'lon'] /= 1000

In [ ]:
stations.loc[stations['state']=='NY'].head()

So now let's use again our lat-lon filter...

In [ ]:
masklong = (stations['lon'] >= minlong) & (stations['lon'] <= maxlong)
stationsLon = (stations.loc[masklong])

masklat = (stationsLon['lat'] >= minlat) &(stationsLon['lat'] <= maxlat)
stationsLonLat = (stationsLon.loc[masklat])

In [ ]:
stationsLonLat

Therefore, in total we have 8 weather stations in the area of Manhattan. By examining again the header of the weather database, we can see that there is a common key between them and the stations dataset; the **"wban"**. So our next goal is to isolate the weather data for the stations that we are interested in. 

In [ ]:
stationsLonLat.wban.unique()

In [ ]:
wbanNY = stationsLonLat.wban.unique()[:2]
print(wbanNY)

In [ ]:
weatherNY = dfWeather.loc[dfWeather['wban'].isin(wbanNY)]

In [ ]:
weatherNY.describe()

So now let's keep in a new dataframe only the attributes that we believe could be useful...

In [ ]:
weatherNY = weatherNY[['wban','year','mo','da','temp','prcp','sndp','fog','rain_drizzle']]
weatherNY.describe()

Everything looks normal, except sndp (check quantiles). Since the value "999.9" does not give any information, we will replace it with "NaN" (In computing, NaN, standing for *not a number*, is a numeric data type value representing an undefined or unrepresentable value.).

In [ ]:
weatherNY.loc[weatherNY.sndp == 999.9, 'sndp'] = nan
weatherNY.describe()

In [ ]:
weatherNY.temp.hist()

In [ ]:
weatherNY.prcp.hist()

From the histograms above, we can verify that our dataset has a normal range of temperature and precipitation values.

So now let's create some new plots using the seaborn library! 

First, I would like to add a new column in my current dataframe with the name of the month. The column **"mo"** has already the number of the month, so I will use the **"calendar.month_abbr"** for the correspondence of each number in the name of the month.

In [ ]:
monthList = []

for i in range(len(weatherNY)):
    monthList.append(calendar.month_abbr[weatherNY.mo[weatherNY.index[i]]])

weatherNY["month"] = monthList

In [ ]:
weatherNY

In [ ]:
def show_weather_stats(dataset, variable, ticks):
    # Initialize a grid of plots with an Axes for each walk
    grid = sns.FacetGrid(dataset,col="month", hue="month", col_wrap=4, size=3)

    # Draw a line plot to show the trajectory of each random walk
    grid.map(plt.plot, "da", variable, marker="o") 

    # Adjust the tick positions and labels
    grid.set(yticks=ticks)

    # Adjust the arrangement of the plots
    grid.fig.tight_layout(w_pad=1)
    plt.show()

In [ ]:
# Choose to plot the precipitation values within the range [0,3]
show_weather_stats(weatherNY, "prcp", [0,3])

Each plot has lines that unites randomly the various points. This usually happens when our dataset is not sorted chronologically. Our next goal is to modify records' order based on the date they refer to.

In [ ]:
dateList = []

for i in range(len(weatherNY)):
    date1 = datetime.date(year=weatherNY.year[weatherNY.index[i]],day=weatherNY.da[weatherNY.index[i]],month=weatherNY.mo[weatherNY.index[i]])
    #date2 = datetime.datetime.combine(date1,datetime.time())
    dateList.append(date1)

In [ ]:
# Add a new column with the date
weatherNY["date"] = dateList

In [ ]:
# Sort values based on the date
weatherNYSorted = weatherNY.sort_values(by='date')
weatherNYSorted.head()

In [ ]:
# Redo the plot
show_weather_stats(weatherNYSorted, "prcp", [0,1,2,3])

In [ ]:
# Make a plot for the snow 
show_weather_stats(weatherNYSorted, "sndp", [0, 10, 20])

In [ ]:
# Finally, make a plot for the temperature values
show_weather_stats(weatherNYSorted, "temp", [0, 10, 20, 40, 60, 80, 90, 100])

### Add weather information to our initial dataset

In [ ]:
# Create a new column "date" in the initial dataset (with lags)
smerged_lagged_s1['date']=[d.date() for d in smerged_lagged_s1.index]

In [ ]:
smerged_lagged_s1.head()

In [ ]:
# Remove index (but keep the datetime information) so that the merging process will be more clear later.
smerged_lagged_s1=smerged_lagged_s1.reset_index()
smerged_lagged_s1.head()

In [ ]:
# Merge two datasets using the "date" as the key 
result=pd.merge(smerged_lagged_s1, weatherNYSorted, on='date', how='inner') # how: inner -> use intersection of keys from both frames, similar to a SQL inner join

In [ ]:
# Set again the datetime as index
result=result.set_index('datetime')

In [ ]:
# Save the final result 
result.to_csv('NYC_taxi_weather.csv')

As you can see, weather information has been included in our dataset (for 2016 only)

### Isolation of 2016 Taxi Data

In [ ]:
TaxiData2016 = result.iloc[result.index.year == 2016]
TaxiData2016.head()

In [ ]:
TaxiData2016.corr()